In [10]:
from langchain.llms import CTransformers
import os 

## Loading LLM - 
we can use huggingface library to directly get the opensource LLM model, Since I have downloaded BIN file
of the llama-2-7b quantized model I can use that directly
To load these LLM model from bin file , you will need CTransformers library
llm.predict will help us predict output of LLMS

In [14]:
llm = CTransformers(
    model='model/llama-2-7b-chat.ggmlv3.q4_0.bin',
    model_type = 'llama',
    config={'max_new_tokens':100,'temperature':0.8}
    )

In [16]:
text = 'what is the capital of India'
print(llm.predict(text))

?
 Hinweis: The answer can be found in the question.
The capital of India is New Delhi.


## Prompt Template -
It helps us to create a template of prompt if we want to pass any paramters along with it . In below example
I have created a template to tell me the capital of a country and I can pass country name in runtime.
LLM chains are used to map the prompt template to runtime input variables

In [20]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [19]:
prompt_template = PromptTemplate(input_variables=['country'], template= 'Tell me the capital of {country}')

prompt_template.format(country = 'Pakistan')

'Tell me the capital of Pakistan'

In [21]:
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.run('Pakistan')

"?\n everybody knows that the capital of Pakistan is Islamabad.\nWell, I'm here to tell you that it's not! The correct answer is actually Karachi. That's right, the largest city in Pakistan and the financial hub of the country is also its capital! But why does everyone think Islamabad is the capital? Well, it's because the government of Pakistan has made a bit of a mess of things over the years, moving the capital from Karachi to"

## Simple Sequential Chain - 
Using this we can pass output of 1 chain as an input of another chain and we can execute multiple chain sequentially.
In below example I am actually creating 2 chains, 1 to get capital as output and i will be using that output in next 
chain input to get 'best places to eat'.

In [23]:
capital_prompt = PromptTemplate(input_variables=['country'],template='Please tell me the capital of {country}')
capital_chain = LLMChain(llm=llm, prompt=capital_prompt)
#capital_chain.run('Bangladesh')

In [24]:
best_places = PromptTemplate(input_variables=['capital'], template='Tell me best places to eat at {capital}')
best_places_chain = LLMChain(llm=llm, prompt= best_places)

In [25]:
from langchain.chains import SimpleSequentialChain
f_chain = SimpleSequentialChain(chains=[capital_chain, best_places_chain])
f_chain.run('India')

' Here are some of the best places to eat in Delhi:\n\n1. Paratha Gali: This street in Old Delhi is famous for its parathas, which are crispy flatbreads stuffed with a variety of fillings such as potatoes, cauliflower, and peas.\n2. Chandni Chowk: This bustling market in Old Delhi offers a wide range of street food options, including chaat,'

## Sequential Chain 
In Sequential chain we create 'Output key' for each chain to display what output we want to be used as an input
in another chain
Below you can see I have inserted output key as capital, as I wanted to fetch capital from 1st chain
Finally in SequentialChain , addition to chains I am passing input variables aand output variables
when you run the chain you can see sequential running of each chain in parameters

In [27]:
capital_prompt = PromptTemplate(input_variables=['country'],template='Please tell me the capital of {country}')
capital_chain = LLMChain(llm=llm, prompt=capital_prompt, output_key= 'capital')

In [28]:
best_places = PromptTemplate(input_variables=['capital'], template='Tell me best places to eat at {capital}')
best_places_chain = LLMChain(llm=llm, prompt= best_places, output_key= 'best_places')

In [29]:
from langchain.chains import SequentialChain
chain = SequentialChain(
    chains=[capital_chain, best_places_chain],
    input_variables=['country'],
    output_variables=['capital','best_places'])


In [32]:
chain('India')

{'country': 'India',
 'capital': ', it is Delhi right? Wrong! The correct answer is New Delhi. Unterscheidung between Delhi and New Delhi can be a bit tricky as they are often used interchangeably. Delhi is the metropolitan city that includes Old Delhi, New Delhi, and other nearby areas, while New Delhi is a specific area within Delhi that serves as the capital city of India.\nNew Delhi was designed by Edwin Lutyens and built in the early',
 'best_places': " 20th century as the new capital of India. It is known for its colonial-era architecture, including the Rashtrapati Bhavan (the President's residence), the Parliament House, and Connaught Place.\nWhen it comes to food, New Delhi offers a diverse range of options, from traditional Indian cuisine to international flavors. Here are some of the best places to eat in New Delhi:\n1. Hari Puttar - This"}